In [12]:
import astropy.units as u
import numpy as np
from astropy.io import ascii

import naima
from naima.models import ExponentialCutoffPowerLaw, InverseCompton, Synchrotron

## Read data

soft_xray1 = ascii.read("mos1+mos2_pwn_nh-corr.dat")
#soft_xray2 = ascii.read("chandra_pwn_nh-corr.dat")
vhe = ascii.read("HAWC_J1826-128.dat")

## Model definition

PionDecay_ECPL_p0 = np.array((46, 2.34, np.log10(80.0)))
PionDecay_ECPL_labels = ["log10(norm)", "index", "log10(cutoff)"]

# Prepare an energy array for saving the particle distribution
proton_energy = np.logspace(-3, 2, 50) * u.TeV


def PionDecay_ECPL(pars, data):
    amplitude = 10 ** pars[0] / u.TeV
    alpha = pars[1]
    e_cutoff = 10 ** pars[2] * u.TeV
#    B = pars[3] * u.uG
    B = 12 * u.uG

    ECPL = naima.models.ExponentialCutoffPowerLaw(
        amplitude, 30 * u.TeV, alpha, e_cutoff
    )
    PP = naima.models.PionDecay(ECPL, nh=500.0 * u.cm ** -3)
    SYN = Synchrotron(ECPL, B=B)

    model = PP.flux(data, distance=3.5 * u.kpc) + SYN.flux(data, distance=3.5 * u.kpc)
    # Save a realization of the particle distribution to the metadata blob
    proton_dist = PP.particle_distribution(proton_energy)
    # Compute the total energy in protons above 1 TeV for this realization
    Wp = PP.compute_Wp(Epmin=1 * u.TeV)

    # Return the model, proton distribution and energy in protons to be stored
    # in metadata blobs
    return model, (proton_energy, proton_dist), Wp


def PionDecay_ECPL_lnprior(pars):
    logprob = (
        naima.uniform_prior(pars[0], 0.0, np.inf)
        + naima.uniform_prior(pars[1], -1, 5)
    )
    return logprob

## Prior definition


#def lnprior(pars):
#    """
#    Return probability of parameter values according to prior knowledge.
#    Parameter limits should be done here through uniform prior ditributions
#    """
#    # Limit norm and B to be positive
#    logprob = (
#        naima.uniform_prior(pars[0], 0.0, np.inf)
#        + naima.uniform_prior(pars[1], -1, 5)
#        + naima.uniform_prior(pars[3], 0, np.inf)
#    )
#
#    return logprob


if __name__ == "__main__":

    ## Set initial parameters and labels
    # Estimate initial magnetic field and get value in uG
#    B0 = 2 * naima.estimate_B(soft_xray1, vhe).to("uG").value
    #B0 = 5

#    p0 = np.array((33, 2.5, np.log10(48.0), B0))
#    B0 = 2 * naima.estimate_B(soft_xray1, vhe).to("uG").value
    labels = ["log10(norm)", "index", "log10(cutoff)", "B"]

    ## Run sampler
    sampler, pos = naima.run_sampler(
        data_table=[soft_xray1, vhe],
        p0=PionDecay_ECPL_p0,
        labels=labels,
        model=PionDecay_ECPL,
        prior=PionDecay_ECPL_lnprior,
        nwalkers=32,
        nburn=100,
        nrun=20,
        threads=4,
        prefit=True,
        interactive=False,
    )

    ## Save run results to HDF5 file (can be read later with naima.read_run)
    naima.save_run("J1826_HAWC_PP", sampler)
    
    ## Diagnostic plots
    naima.save_diagnostic_plots(
        "J1826_HAWC_PP",
        sampler,
        sed=True,
        blob_labels=["Spectrum", "$W_e$($E_e>1$ TeV)"],
    )
    naima.save_results_table("J1826_HAWC_PP", sampler)
    
    plt = naima.plot_fit(sampler, label="Eel Pulsar Wind Nebula SED", sed=True, confs=[3,1], e_range=[0.00001*u.eV, 100*u.TeV])
    
    plt.savefig('J1826_SED_HAWC_PP.pdf', format='pdf', bbox_inches='tight')
    
    plt.show()

INFO: Finding Maximum Likelihood parameters through Nelder-Mead fitting... [naima.core]
INFO:    Initial parameters: [41.36592918  2.34        1.90308999] [naima.core]
INFO:    Initial lnprob(p0): -90.279 [naima.core]
INFO:    New ML parameters : [40.79451229  1.72791894  2.20022721] [naima.core]
INFO:    Maximum lnprob(p0): -32.041 [naima.core]
Burning in the 32 walkers with 100 steps...

Progress of the run: 0 percent (0 of 100 steps)
                           --log10(norm)-- -----index----- -log10(cutoff)-
  Last ensemble median :      40.8            1.73             2.2      
  Last ensemble std    :      0.182          0.00757         0.0127     
  Last ensemble lnprob :  avg: -76.565, max: -32.048

Progress of the run: 5 percent (5 of 100 steps)
                           --log10(norm)-- -----index----- -log10(cutoff)-
  Last ensemble median :      40.8            1.73             2.2      
  Last ensemble std    :      0.128          0.0101          0.0168     
  Last ensemble


Progress of the run: 20 percent (4 of 20 steps)
                           --log10(norm)-- -----index----- -log10(cutoff)-
  Last ensemble median :      40.8            1.64            2.25      
  Last ensemble std    :      0.123           0.513           0.431     
  Last ensemble lnprob :  avg: -33.098, max: -32.008

Progress of the run: 25 percent (5 of 20 steps)
                           --log10(norm)-- -----index----- -log10(cutoff)-
  Last ensemble median :      40.8            1.67            2.25      
  Last ensemble std    :      0.131           0.542           0.483     
  Last ensemble lnprob :  avg: -33.168, max: -32.008

Progress of the run: 30 percent (6 of 20 steps)
                           --log10(norm)-- -----index----- -log10(cutoff)-
  Last ensemble median :      40.8            1.67            2.31      
  Last ensemble std    :      0.128           0.523           0.479     
  Last ensemble lnprob :  avg: -33.148, max: -32.008

Progress of the run: 35 percen

INFO: Plotting chain of parameter index... [naima.analysis]
INFO: ----------------------index-----------------------
          index = $1.7^{+0.6}_{-0.4}$ [naima.plot]
INFO: Plotting chain of parameter log10(cutoff)... [naima.analysis]
INFO: ------------------log10(cutoff)-------------------
          log10(cutoff) = $2.3^{+0.6}_{-0.3}$
                 cutoff = $200^{+700}_{-100}$ [naima.plot]
INFO: Plotting corner plot... [naima.analysis]
INFO: Plotting Spectrum... [naima.analysis]


/Users/danielburgess/anaconda3/lib/python3.7/site-packages/naima/analysis.py:160: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
  f.clf()
/Users/danielburgess/anaconda3/lib/python3.7/site-packages/naima/analysis.py:160: UserWarning: Attempting to set identical left == right == 1.0 results in singular transformations; automatically expanding.
  f.clf()


INFO: Plotting $W_e$($E_e>1$ TeV)... [naima.analysis]
INFO: Plotting Model output 2... [naima.analysis]
INFO: Saving results table in J1826_HAWC_PP_results.ecsv [naima.analysis]


IndexError: index 3 is out of bounds for axis 1 with size 3

NameError: name 'plt' is not defined